In [59]:



class Morph:
    
    def __init__(self, surface, base, pos, pos1):
        '''初期化'''
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        '''オブジェクトの文字列表現'''
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(self.surface, self.base, self.pos, self.pos1)


def neco_lines():
    
    with open('neko.txt.cabocha') as fcabo:

        morphs = []
        for line in fcabo:

            # 1文の終了判定
            if line == 'EOS\n':
                yield morphs
                morphs = []

            else:
                # *はスキップ
                if line[0] == '*':
                    continue

                # 表層形はtab区切り、それ以外は','区切りでバラす
                cols = line.split('\t')
                res_cols = cols[1].split(',')

                # Morph作成、リストに追加
                morphs.append(Morph(
                    cols[0],        # surface
                    res_cols[6],    # base
                    res_cols[0],    # pos
                    res_cols[1]     # pos1
                ))

        raise StopIteration


# 1文ずつリスト作成
for i, morphs in enumerate(neco_lines(), 1):

    # 3文目を表示
    if i == 3:
        for morph in morphs:
            print(morph)
        break

surface[　]	base[　]	pos[記号]	pos1[空白]
surface[吾輩]	base[吾輩]	pos[名詞]	pos1[代名詞]
surface[は]	base[は]	pos[助詞]	pos1[係助詞]
surface[猫]	base[猫]	pos[名詞]	pos1[一般]
surface[で]	base[だ]	pos[助動詞]	pos1[*]
surface[ある]	base[ある]	pos[助動詞]	pos1[*]
surface[。]	base[。]	pos[記号]	pos1[句点]


In [24]:
#40に加えて，文節を表すクラスChunkを実装せよ．
#このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），
#係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．
#さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，
#8文目の文節の文字列と係り先を表示せよ．
#第5章の残りの問題では，ここで作ったプログラムを活用せよ．


In [3]:
#係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．
#ただし，句読点などの記号は出力しないようにせよ

In [4]:
#名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．
#ただし，句読点などの記号は出力しないようにせよ．



In [5]:
#与えられた文の係り受け木を有向グラフとして可視化せよ．
#可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，
#Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

